In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist

(X_train, _), (_, _) = mnist.load_data()
X_train = X_train.astype('float32') / 255.0
X_train = X_train.reshape(-1, 28 * 28)

random_dim = 100
img_dim = 28 * 28
batch_size = 128

# Generator Model
generator = Sequential([
    Dense(256, activation='relu', input_dim=random_dim),
    Dense(512, activation='relu'),
    Dense(1024, activation='relu'),
    Dense(img_dim, activation='sigmoid')  # Output matches the flattened image
])

# Discriminator Model
discriminator = Sequential([
    Dense(1024, activation='relu', input_dim=img_dim),
    Dense(512, activation='relu'),
    Dense(256, activation='relu'),
    Dense(1, activation='sigmoid')  # Output is a single probability (real or fake)
])

# Compile Discriminator
discriminator.compile(optimizer=Adam(learning_rate=0.0002, beta_1=0.5), loss='binary_crossentropy', metrics=['accuracy'])
discriminator.trainable = False

# GAN Model (Generator + Discriminator)
gan = Sequential([generator, discriminator])
gan.compile(optimizer=Adam(learning_rate=0.0002, beta_1=0.5), loss='binary_crossentropy')


11490434/11490434 [==============================] - 2s 0us/step


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

data = pd.read_csv('ndvi.csv')
scaler = MinMaxScaler(feature_range=(0, 1))
data['value'] = scaler.fit_transform(data['value'].values.reshape(-1, 1))

def create_sequences(data, timesteps):
    X, y = [], []
    for i in range(len(data) - timesteps):
        X.append(data[i:i+timesteps])  # Previous timesteps
        y.append(data[i+timesteps])   # Target: next value
    return np.array(X), np.array(y)

timesteps = 10
values = data['value'].values
X, y = create_sequences(values, timesteps)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

model = Sequential([
    LSTM(50, activation='relu', input_shape=(X_train.shape[1], 1)),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
3/3 [==============================] - 1s 9ms/step - loss: 0.2153
Epoch 2/10
3/3 [==============================] - 0s 8ms/step - loss: 0.1917
Epoch 3/10
3/3 [==============================] - 0s 8ms/step - loss: 0.1653
Epoch 4/10
3/3 [==============================] - 0s 7ms/step - loss: 0.1441
Epoch 5/10
3/3 [==============================] - 0s 8ms/step - loss: 0.1236
Epoch 6/10
3/3 [==============================] - 0s 8ms/step - loss: 0.1081
Epoch 7/10
3/3 [==============================] - 0s 6ms/step - loss: 0.1040
Epoch 8/10
3/3 [==============================] - 0s 6ms/step - loss: 0.1026
Epoch 9/10
3/3 [==============================] - 0s 7ms/step - loss: 0.1061
Epoch 10/10
3/3 [==============================] - 0s 8ms/step - loss: 0.1062
